In [1]:
# pip install twilio

# import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, PHONE_NUMBER, API_KEY_WAPI
# import time

# from requests import Request,Session
# from requests.exceptions import ConnectionError,Timeout,TooManyRedirects
import json

import pandas as pd
import requests
# from bs4 import BeautifulSoup
from tqdm import tqdm
from time import sleep

# from datetime import datetime

In [2]:
query = "San Salvador de Jujuy"
api_key = API_KEY_WAPI

url_clima = f"http://api.weatherapi.com/v1/forecast.json?key={api_key}&q={query}&days=1&aqi=no&alerts=no"

response = requests.get(url_clima).json()
# Ver la estructura que almacena la variable response en: 'estructura.json'

In [3]:
# Llaves principales de la estructura del JSON. 
print(response.keys())

dict_keys(['location', 'current', 'forecast'])


In [4]:
# Llaves principales de la estructura de segundo nivel dentro de "forecast"
print(response['forecast']['forecastday'][0].keys())

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])


In [5]:
# Dentro de la llave 'hour' tenemos una lista de diccionarios correspondientes al pronostico en cada hora del dia, es decir 24 diccionarios 
print(len(response['forecast']['forecastday'][0]['hour']))

24


In [6]:
# TOMANDO COMO EJEMPLO EL DECIMO DICCIONARIO (10:00hs)
# Obtener fecha
print(response['forecast']['forecastday'][0]['hour'][10]['time'].split()[0])
# Obtener hora
print(response['forecast']['forecastday'][0]['hour'][10]['time'].split()[1].split(':')[0])
# Obtener condicion
print(response['forecast']['forecastday'][0]['hour'][10]['condition']['text'])
# Obtener temperatura
print(response['forecast']['forecastday'][0]['hour'][10]['temp_c'])
# Obtener respuesta a llovera? 1=si 0=no
print(response['forecast']['forecastday'][0]['hour'][10]['will_it_rain'])
# Obtener probabilidad de lluvia
print(response['forecast']['forecastday'][0]['hour'][10]['chance_of_rain'])

2024-01-12
10
Light drizzle
18.4
1
100


In [7]:
def get_forecast(response,i):
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[
        1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    temperatura = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    lluvia = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_lluvia = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    return fecha,hora,condicion,temperatura,lluvia,prob_lluvia

In [8]:
datos = []
for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])), colour='green'):    
    datos.append(get_forecast(response,i))    
    #sleep(0.5)    

100%|██████████| 24/24 [00:00<00:00, 45569.62it/s]


In [9]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','Prob_lluvia']
df = pd.DataFrame(datos,columns=col)

In [10]:
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,Prob_lluvia
0,2024-01-12,0,Fog,16.2,1,75
1,2024-01-12,1,Mist,15.2,0,0
2,2024-01-12,2,Mist,13.8,0,0
3,2024-01-12,3,Partly cloudy,13.1,0,0
4,2024-01-12,4,Partly cloudy,12.8,0,0
5,2024-01-12,5,Mist,12.3,0,0
6,2024-01-12,6,Mist,12.4,0,0
7,2024-01-12,7,Mist,15.8,0,0
8,2024-01-12,8,Patchy rain possible,17.7,1,89
9,2024-01-12,9,Light drizzle,18.1,1,100


In [11]:
df.dtypes

Fecha           object
Hora             int64
Condicion       object
Temperatura    float64
Lluvia           int64
Prob_lluvia      int64
dtype: object

In [12]:
df_rain = df[(df['Lluvia']==1) & (df['Hora']>=6)]
df_rain = df_rain[['Hora','Condicion']]
df_rain = df_rain.set_index('Hora')
df_rain

,Condicion
Hora,
8,Patchy rain possible
9,Light drizzle
10,Light drizzle
11,Light rain shower
12,Light rain shower
13,Light rain shower
14,Light rain shower
15,Light rain shower
16,Light drizzle


In [13]:
# MENSAJE
mensaje = f"Hola!\nEl pronostico de lluvia para hoy {df['Fecha'][0]} en {query} es:\n{str(df_rain)}"
mensaje

'Hola!\nEl pronostico de lluvia para hoy 2024-01-12 en San Salvador de Jujuy es:\n                 Condicion\nHora                      \n8     Patchy rain possible\n9            Light drizzle\n10           Light drizzle\n11       Light rain shower\n12       Light rain shower\n13       Light rain shower\n14       Light rain shower\n15       Light rain shower\n16           Light drizzle\n17       Light rain shower\n18           Partly cloudy\n23                    Mist'

In [14]:
# NUMERO
PHONE_NUMBER

'+1 720 583 9694'

In [15]:
#import os
#from twilio.rest import Client


# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
    .create(
        body=mensaje,
        from_=PHONE_NUMBER,
        to='+543886857117'
    )

print('Mensaje enviado! ',message.sid)

Mensaje enviado!  SM912a040f717add9ca3dbafacd161664e
